In [12]:
using JLD
using Distributions
using LinearAlgebra
using Random
using WAV
using Plots
using ImageCore
using GraphPPL
using ReactiveMP
using Optim
using AIDA
using Parameters
import ProgressMeter

In [ ]:
function init_environment(; from_scratch=false)
    
    if !from_scratch
        # read context memory 
        context_priors = [JLD.load(file) for file in get_sounds_fn("../src/memory/context")]    

        # read processed auido
        test_jlds = get_files("sound/AIDA/separated_jld/test/");
        
        
    end
    
end

In [ ]:
function init_agent(; from_scratch=false)
    # read preferences
    preferences = JLD.load("../src/memory/preferences.jld")
    
    # read neural params of trained NN?
end

In [19]:
function preference_learning(preferences, context_num; augment_data=true)
    data_x, data_y = augment_data ? get_learning_data(preferences, context_num) : (preferences["gains"], preferences["appraisals"])
    function f(params)
        fe = inference_flow_classifier(Float64.(data_y), [data_x[k,:] for k=1:size(data_x,1)], params)
    end
    res = optimize(f, randn(nr_params(model)), LBFGS(), Optim.Options(store_trace = true, show_trace = true, f_tol=1e-12), autodiff=:forward)
end

preference_learning (generic function with 1 method)

In [ ]:
function listen()
    # 
end

In [ ]:
function act()
    #
end